In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

# 1. SETUP
# Path to your TRAINED model
model_path = r"C:\Users\Ahmed\OneDrive\Desktop\NLP\NLP_Project_Propaganda\models\arabert_optimized"
# Name of the ORIGINAL tokenizer (we download this to avoid local file errors)
tokenizer_name = "aubmindlab/bert-base-arabertv02"

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"⏳ Loading Tokenizer from {tokenizer_name}...")
# FIX: Load tokenizer from the web, but model from local folder
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

print(f"⏳ Loading Optimized Model from {model_path}...")
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
print("✅ System Ready!")

# 2. Define the Prediction Function
def predict_propaganda(text):
    # Prepare text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    
    # Get Prediction
    with torch.no_grad():
        logits = model(**inputs).logits
        probabilities = torch.softmax(logits, dim=1)
    
    # Get Scores
    # Class 0 = Non-Propaganda, Class 1 = Propaganda
    non_prop_score = probabilities[0][0].item()
    prop_score = probabilities[0][1].item()
    
    return {
        "Non-Propaganda (محايد)": non_prop_score, 
        "Propaganda (دعاية)": prop_score
    }

# 3. Create the Interface
demo = gr.Interface(
    fn=predict_propaganda,
    inputs=gr.Textbox(lines=3, placeholder="أدخل النص هنا... (Enter Arabic text here)"),
    outputs=gr.Label(num_top_classes=2),
    title="🛡️ Arabic Propaganda Detector (AraBERT)",
    description="This AI detects if a text regarding the Gaza war contains propaganda techniques.",
    examples=[
        ["قامت القوات الإسرائيلية بقصف مدرسة تؤوي نازحين في غزة"],  
        ["العدو الصهيوني الغاشم يرتكب مجازر وحشية ضد المدنيين الأبرياء"], 
    ]
)

# 4. Launch!
demo.launch(share=True)

⏳ Loading Tokenizer from aubmindlab/bert-base-arabertv02...
⏳ Loading Optimized Model from C:\Users\Ahmed\OneDrive\Desktop\NLP\NLP_Project_Propaganda\models\arabert_optimized...
✅ System Ready!
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
